# Recommendation system for arXiv manuscripts by Peter Boross

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import json
import arxiv
import sqlite3
import urllib.request as libreq
import re
from collections import Counter
import unidecode
import itertools
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import ShuffleSplit

### Helper functions

In [3]:
def get_authors_LastF(authors):
    r = []
    for author in authors:
        if len(author[1]) == 0: r.append(unidecode.unidecode(author[0]))
        else: r.append(unidecode.unidecode(author[0]+author[1][0]))    
    return ' '.join(r)

def get_authors_LastF_2(authors):
    r = []
    for authorv in authors:
        author = authorv['name'].split(' ')
        r.append(unidecode.unidecode(author[-1]+author[0][0]))    
    return ' '.join(r)

### Define categories

In [4]:
categories = {'cond-mat', 'cond-mat.mes-hall', 'quant-ph', 'cond-mat.supr-con', 'cond-mat.mtrl-sci', 'cond-mat.str-el', 'cond-mat.other'}

### Load manuscripts from arXiv JSON by Kaggle

In [4]:
articles = []

with open("data/arxiv-metadata-oai-snapshot.json", "r") as f:
    for l in f:
        d = json.loads(l)
        if categories & set(d['categories'].split(' ')):
            d['authors_LastF'] = get_authors_LastF(d['authors_parsed'])
            articles.append(d)

articles_df = pd.DataFrame().from_records(articles)

print('number of papers =',len(articles))

number of papers = 315071


### Find manuscript of the authors

In [5]:
authors = ['BorossP','OroszlanyL','PalyiA','AsbothJ','SzechenyiG']

ids = list(articles_df[articles_df["authors_LastF"].str.contains('|'.join(authors))]['id'])

print('number of papers of the authors =',len(ids))

number of papers of the authors = 90


### Find cited papers by prophy.science

In [6]:
refs=[]

for id in ids:
    with libreq.urlopen('https://www.prophy.science/api/arxiv/' + id) as url:
        refs1paper = json.loads(url.read())
    refs.extend([ref['arxivId'] for ref in refs1paper['references'] if ref['arxivId'] != None])

refscounted = sorted(Counter(refs).items(), key=lambda pair: pair[1], reverse=True)
refs = [entry[0] for entry in refscounted]
counts = [entry[1] for entry in refscounted]
refscounteddict = dict(zip(refs, counts))

print('number of cited papers of the authors =',len(refscounted))

number of cited papers of the authors = 1501


### Make training dataset and write it out

In [7]:
cited_df = articles_df[articles_df['id'].isin(refs)][['abstract','title','authors_LastF','id']].replace(refscounteddict).rename(columns = {'id': 'citation', 'authors_LastF': 'authors'})
cited_df['cited'] = True

print('number of cited papers which in the selected categories =',len(cited_df))

notcited_df = articles_df[articles_df['id'].isin(refs) == False][['abstract','title','authors_LastF']].sample(n = 10*len(cited_df)).rename(columns = {'authors_LastF': 'authors'})
notcited_df['citation'] = 0
notcited_df['cited'] = False

print('number of non-cited papers which in the selected categories =',len(notcited_df))

all_df = pd.concat([cited_df, notcited_df])

all_df.to_csv('data/all_df.csv')

number of cited papers which in the selected categories = 1458
number of non-cited papers which in the selected categories = 14580


### Split to X and y

In [8]:
#all_df = pd.read_csv('data/all_df.csv', index_col=0)

X = all_df[['authors','title','abstract']]
y = list(all_df['cited'])

### Build the model and make cross validation

In [9]:
authors_feature = 'authors'
authors_transformer = CountVectorizer(lowercase=False, max_features = 1000)

title_feature = 'title'
title_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1,3), max_features = 2000)

abstract_feature = 'abstract'
abstract_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1,3), max_features = 5000)

preprocessor = ColumnTransformer(
    transformers=[
        ('authors', authors_transformer, authors_feature),
        ('title', title_transformer, title_feature),
        ('abstract', abstract_transformer, abstract_feature)
    ])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', MultinomialNB())])

scores = cross_validate(pipeline, X, y, cv=ShuffleSplit(n_splits=5),
                        scoring=('accuracy', 'precision', 'recall' , 'roc_auc'),
                        return_train_score=True)

print('train accuracy = ',"{:.1f}%".format(100*np.mean(scores['train_accuracy'])),'test accuracy =',"{:.1f}%".format(100*np.mean(scores['test_accuracy'])))
print('train precision = ',"{:.1f}%".format(100*np.mean(scores['train_precision'])),'test precision =',"{:.1f}%".format(100*np.mean(scores['test_precision'])))
print('train recall = ',"{:.1f}%".format(100*np.mean(scores['train_recall'])),'test recall =',"{:.1f}%".format(100*np.mean(scores['test_recall'])))
print('train roc_auc = ',"{:.1f}%".format(100*np.mean(scores['train_roc_auc'])),'test roc_auc =',"{:.1f}%".format(100*np.mean(scores['test_roc_auc'])))

train accuracy =  94.0% test accuracy = 93.5%
train precision =  65.7% test precision = 63.6%
train recall =  72.1% test recall = 68.4%
train roc_auc =  95.6% test roc_auc = 93.3%


### Fit the model and save it

In [10]:
pipeline.fit(X, y);

filename = 'data/model.sav'
pickle.dump(pipeline, open(filename, 'wb'))

### Make a query and predict

In [67]:
filename = 'data/model.sav'
pipeline = pickle.load(open(filename, 'rb'))

delta = timedelta(days=1)
catstr = '+OR+'.join(['cat:'+x for x in categories])
client = arxiv.Client()
latest = False
n_query = 50
start_query = 0
last_query = n_query
df = pd.DataFrame(columns = ['id','authors', 'title', 'abstract'])

while last_query == n_query:
    feedparser = client._parse_feed(url='http://export.arxiv.org/api/query?search_query='+catstr+'&start='+str(start_query)+'&max_results='+str(n_query)+'&sortBy=submittedDate')
    last_query=0
    for entry in feedparser.entries:
        if not(latest): latest = datetime.strptime(entry.published[0:10],'%Y-%m-%d')
        if latest - datetime.strptime(entry.published[0:10],'%Y-%m-%d') < delta:
            last_query+=1
            df = df.append({
                'id' : entry.id,
                'authorsFull' : [author['name'] for author in entry.authors],
                'authors' : get_authors_LastF_2(entry.authors),
                'title' : entry.title.rstrip(),
                'abstract' : entry.summary.rstrip(),
                'published': datetime.strptime(entry.published[0:10],'%Y-%m-%d')
                            }, ignore_index = True)
    start_query += n_query

Xnew = df[['authors','title','abstract']]

df['rating'] = [x[1] for x in pipeline.predict_proba(Xnew)]

In [45]:
df[['id','published','authorsFull','title','abstract','rating']].sort_values(by=['rating'],ascending=False).head(5)

,id,published,authorsFull,title,abstract,rating
37,http://arxiv.org/abs/2105.14864v1,2021-05-31,"[A. Rossi, N. W. Hendrickx, A. Sammak, M. Veld...",Single-Hole Pump in Germanium,Single-charge pumps are the main candidates fo...,0.806654
50,http://arxiv.org/abs/2105.14763v1,2021-05-31,"[Niels John, Adrian Del Maestro, Bernd Rosenow]",Robustness of Helical Edge States Under Edge R...,The helical edge states of time-reversal invar...,0.789467
39,http://arxiv.org/abs/2105.14842v1,2021-05-31,"[Thomas Simons, Alessandro Romito, Dganit Meidan]",Relation between scattering matrix topological...,We analyze the conductance of a one-dimensiona...,0.679321
32,http://arxiv.org/abs/2105.14919v1,2021-05-31,"[Yoonseok Hwang, Jun-Won Rhim, Bohm-Jung Yang]",Flat bands with band crossings enforced by sym...,Flat bands have band crossing points with othe...,0.291665
28,http://arxiv.org/abs/2105.14941v1,2021-05-31,"[Huan-Wen Wang, Bo Fu, Shun-Qing Shen]",Helical Symmetry Breaking and Quantum Anomaly ...,Helical symmetry of massive Dirac fermions is ...,0.218304


In [1]:
conn = sqlite3.connect('data/manuscripts.db')
c = conn.cursor()

c.execute('CREATE TABLE IF NOT EXISTS manuscipts (id, published, title, abstract, rating)')
conn.commit()

NameError: name 'sqlite3' is not defined

In [76]:
dfs = df[['id','published','title','abstract','rating']]

dfs.to_sql('MANUSCRIPTS', conn, if_exists='replace', index = False)

In [71]:
c.execute('''  
SELECT id FROM MANUSCRIPTS
          ''')